In [8]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from src.model import DogClassifier
from src.tools import MetricStorer, seed_everything
from src.config import DATASET_PATHS, CNFG, SAVE_METRICS_PATH, SAVE_WEIGHTS_PATH
from src.loader import WoofDataset, train_transforms, val_transforms

In [9]:
seed_everything(CNFG['seed'])

In [10]:
labels = pd.read_csv(DATASET_PATHS['labels'])
labels.sample(5)

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid
5687,train/n02099601/n02099601_1639.JPEG,n02099601,n02099601,n02099601,n02099601,n02099601,False
937,train/n02115641/n02115641_14534.JPEG,n02115641,n02115641,n02115641,n02115641,n02115641,False
7727,train/n02093754/n02093754_8005.JPEG,n02093754,n02093754,n02093754,n02093754,n02099601,False
7635,train/n02093754/n02093754_1536.JPEG,n02093754,n02093754,n02096294,n02093754,n02093754,False
12614,val/n02093754/n02093754_590.JPEG,n02093754,n02093754,n02093754,n02093754,n02093754,True


In [3]:
labels = labels[['path', 'noisy_labels_0']]

In [4]:
train_labels = labels[:9025]
val_labels = labels[9025:]

In [5]:
train_dataset = WoofDataset(files=train_labels['path'].to_list(),
                            labels=train_labels['noisy_labels_0'].to_list(),
                            data_path=DATASET_PATHS['main_path'],
                            transforms=train_transforms,
                            mode='train')

val_dataset = WoofDataset(files=val_labels['path'].to_list(),
                            labels=val_labels['noisy_labels_0'].to_list(),
                            data_path=DATASET_PATHS['main_path'],
                            transforms=val_transforms,
                            mode='val')

In [6]:
train_loader = DataLoader(train_dataset, batch_size=CNFG['batch_size'], num_workers=CNFG['num_workers'])
val_loader = DataLoader(val_dataset, batch_size=CNFG['batch_size'], num_workers=CNFG['num_workers'])

In [7]:
print(f'Train length - {len(train_dataset)}, val length - {len(val_dataset)}')

Train length - 9025, val length - 3929


In [7]:
model = DogClassifier(CNFG['model_arch'], 10, pretrained=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CNFG['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.3)

storer = MetricStorer(
    metrics=('acc'),
    raw_names=('train_preds', 'test_preds',
                'train_real', 'test_real'),
    folds=('train', 'test'),
    path=SAVE_METRICS_PATH,
    esr=CNFG['esr']
)
